# Breadth Indicators

Breadth Indicators include:

On Balance Volume

McClellan Summation Index
Arms Index (TRIN)

Force Index

Chaikin Oscillator

Up/Down Volume Ratio

Up/Down Volume Spread

Cumulative Volume Index

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

# fix_yahoo_finance is used to fetch data 
import fix_yahoo_finance as yf
yf.pdr_override()

In [ ]:
# input
symbol = 'SPY'
start = '2012-01-01'
end = '2019-01-01'

# Read data 
df = yf.download(symbol,start,end)

# View Columns
df.head()

In [ ]:
df['Adj Close'][1:]

In [ ]:
import talib as ta

## On Balance Volume

In [ ]:
OBV = ta.OBV(df['Adj Close'], df['Volume'])

## McClellan Summation Index Arms Index (TRIN)

https://www.investopedia.com/terms/m/mcclellanoscillator.asp

In [ ]:
import quandl as q

# For NASDAQ
#Advances = q.get('URC/NASDAQ_ADV')['Numbers of Stocks']
#Declines = q.get('URC/nASDAQ_DEC')['Numbers of Stocks']   
#n = Advances - Declines

In [ ]:
Advances = q.get('URC/NYSE_ADV', start_date = "2017-07-27")['Numbers of Stocks']
Declines = q.get('URC/NYSE_DEC', start_date = "2017-07-27")['Numbers of Stocks']  

In [ ]:
adv_vol = q.get("URC/NYSE_ADV_VOL", start_date = "2017-07-27")['Numbers of Stocks']
dec_vol = q.get("URC/NYSE_DEC_VOL", start_date = "2017-07-27")['Numbers of Stocks']

In [ ]:
data = pd.DataFrame()
data['Advances'] = Advances
data['Declines'] = Declines
data['adv_vol'] = adv_vol
data['dec_vol'] = dec_vol

In [ ]:
data['Net_Advances'] = data['Advances'] - data['Declines'] 
data['Ratio_Adjusted'] = (data['Net_Advances']/(data['Advances'] + data['Declines'])) * 1000
data['19_EMA'] = ta.EMA(data['Ratio_Adjusted'], timeperiod=19)
data['39_EMA'] = ta.EMA(data['Ratio_Adjusted'], timeperiod=39)
data['RANA'] = (data['Advances'] - data['Declines']) / (data['Advances'] + data['Declines']) * 1000

In [ ]:
# Finding the TRIN Value
data['ad_ratio'] = data['Advances'].divide(data['Declines'] ) # AD Ratio
data['ad_vol'] = data['adv_vol'].divide(data['dec_vol']) # AD Volume Ratio
data['TRIN'] = data['ad_ratio'].divide(data['adv_vol']) # TRIN Value

In [ ]:
data.head()

## Force Index

In [ ]:
def ForceIndex(data,n):
    ForceIndex=pd.Series(df['Adj Close'].diff(n)* df['Volume'],name='ForceIndex')
    data = data.join(ForceIndex)
    return data

In [ ]:
n=10
ForceIndex = ForceIndex(data,n)
ForceIndex = ForceIndex['ForceIndex']

In [ ]:
fig=plt.figure(figsize=(7,5))
ax=fig.add_subplot(2,1,1)
ax.set_xticklabels([])
plt.plot(df['Adj Close'],lw=1)
plt.title('Market Price Chart')
plt.ylabel('Close Price')
plt.grid(True)
bx=fig.add_subplot(2,1,2)
plt.plot(ForceIndex,'k',lw=0.75,linestyle='-',label='Force Index')
plt.legend(loc=2,prop={'size':9.5})
plt.ylabel('Force Index')
plt.grid(True)
plt.setp(plt.gca().get_xticklabels(),rotation=30)
plt.show()

## Chaikin Oscillator

In [ ]:
def Chaikin(data):
    money_flow_volume = (2 * df['Adj Close'] - df['High'] - df['Low']) / (df['High'] - df['Low']) * df['Volume']  
    ad = money_flow_volume.cumsum()
    Chaikin = pd.Series(ad.ewm(com=(3-1)/2).mean() - ad.ewm(com=(10-1)/2).mean(), name='Chaikin')
    data = data.join(Chaikin)  
    return data

In [ ]:
Chaikin(df)

## Up/Down Volume Ratio

Volume Spread = Up Volume - Down Volume 

In [ ]:
Up = q.get('URC/NYSE_ADV', start_date = "2017-07-27")['Numbers of Stocks']
Down = q.get('URC/NYSE_DEC', start_date = "2017-07-27")['Numbers of Stocks']
Volume_Spread = Up - Down

In [ ]:
Volume_Spread

## Up/Down Volume Spread

Volume Ratio = Up Volume / Down Volume

In [ ]:
Up = q.get('URC/NYSE_ADV', start_date = "2017-07-27")['Numbers of Stocks']
Down = q.get('URC/NYSE_DEC', start_date = "2017-07-27")['Numbers of Stocks']
Volume_Ratio = Up/Down

In [ ]:
Volume_Ratio

## Cumulative Volume Index

https://www.marketinout.com/technical_analysis.php?t=Cumulative_Volume_Index_(CVI)&id=38

In [ ]:
# CVI = Yesterday's CVI + (Advancing Volume - Declining Volume)
data['CVI'] = data['Net_Advances'][1:] + (data['Advances'] - data['Declines']) 

In [ ]:
data.head()